In [1]:


from datasets import load_dataset

from transformers import AutoTokenizer

import numpy as np

import evaluate


from transformers import TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sep2022",ignore_mismatched_sizes=True)




from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sep2022", num_labels=5,ignore_mismatched_sizes=True)






2022-12-02 17:44:44.233700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sep2022 were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expe

In [2]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')


dataset = load_dataset('tweet_eval', 'sentiment')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    do_eval=True,
    evaluation_strategy='epoch',
    output_dir='test_trainer',
    logging_dir='test_trainer',
    logging_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=3,
    learning_rate=1e-05,
    per_gpu_train_batch_size=16,
    per_gpu_eval_batch_size=16,
    load_best_model_at_end=True,
    metric_for_best_model='recall',
)

metric = evaluate.load('recall')  # default metric for sentiment dataset is recall (macro)

trainer = Trainer(
    model=model.cuda(),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

Found cached dataset tweet_eval (/home/fanm/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/fanm/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-5cb7a7b3062bb44b.arrow
Loading cached processed dataset at /home/fanm/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-47ce01b3969745bf.arrow
Loading cached processed dataset at /home/fanm/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-7525ea238acd4515.arrow
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassificati

Epoch,Training Loss,Validation Loss,Recall
1,0.639800,0.584925,0.730964
2,0.505400,0.603142,0.740271
3,0.431000,0.644763,0.741350


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Saving model checkpoint to test_trainer/checkpoint-2851
Configuration saved in test_trainer/checkpoint-2851/config.json
Model weights saved in test_trainer/checkpoint-2851/pytorch_model.bin
The following columns in the ev

TrainOutput(global_step=8553, training_loss=0.5253809056169985, metrics={'train_runtime': 7221.9758, 'train_samples_per_second': 18.948, 'train_steps_per_second': 1.184, 'total_flos': 3.600640220548608e+16, 'train_loss': 0.5253809056169985, 'epoch': 3.0})